In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

C:\Users\kitcoop\Anaconda3\envs\test\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
df = pd.read_csv("testdata/bmi.csv")

In [3]:
df.head()

,height,weight,label
0,142,62,fat
1,142,73,fat
2,177,61,normal
3,187,48,thin
4,153,60,fat


In [4]:
df['height'] = df['height']/200
df['weight'] = df['weight']/100

print(df.head(3))

   height  weight   label
0   0.710    0.62     fat
1   0.710    0.73     fat
2   0.885    0.61  normal


In [5]:
bclass  = {"thin":[1,0,0], "normal":[0,1,0], "fat":[0,0,1]}

In [6]:
df['label_pat'] = df['label'].apply(lambda x : np.array(bclass[x]))

In [7]:
print("2만개의 자료 중 5000개를 테스트 데이터로 사용")
test_csv = df[15000:20000]

2만개의 자료 중 5000개를 테스트 데이터로 사용


In [8]:
test_pat = test_csv[['height', 'weight']]

In [9]:
test_label = test_csv['label_pat']

In [10]:
df.columns

Index(['height', 'weight', 'label', 'label_pat'], dtype='object')

In [11]:
train_pat = df[['height', 'weight']][:15000]

In [12]:
df.columns

Index(['height', 'weight', 'label', 'label_pat'], dtype='object')

In [13]:
train_label = df['label_pat'][0:15000]

In [14]:
train_label.head()

0    [0, 0, 1]
1    [0, 0, 1]
2    [0, 1, 0]
3    [1, 0, 0]
4    [0, 0, 1]
Name: label_pat, dtype: object

In [15]:
train_label.head()

0    [0, 0, 1]
1    [0, 0, 1]
2    [0, 1, 0]
3    [1, 0, 0]
4    [0, 0, 1]
Name: label_pat, dtype: object

In [16]:
len(train_label)

15000

In [17]:
len(train_pat)

15000

In [18]:
train_label = list(train_label)

In [19]:
np.shape(train_label)

(15000, 3)

In [20]:
# 분류 모델
x = tf.placeholder(tf.float32, [None, 2])
y = tf.placeholder(tf.float32, [None, 3])

In [26]:
tf.set_random_seed(123)

In [27]:
test_pat.head()

,height,weight
15000,0.690,0.55
15001,0.760,0.36
15002,0.915,0.72
15003,0.990,0.51
15004,0.745,0.47


In [28]:
test_label = np.array(list(test_label))

In [30]:
w = tf.Variable(tf.random_normal([2, 3]))
b = tf.Variable(tf.random_normal([3]))

h = tf.nn.softmax(tf.matmul(x, w) + b)
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y*tf.log(h) , axis=1))

optimizer = tf.train.GradientDescentOptimizer(1)
train = optimizer.minimize(cross_entropy)

is_correct = tf.equal(tf.argmax(h, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(5000): # sample 5000
        #i = (step * 100) % 15000
        batch_idx = np.random.choice(15000, 100)
        batch_x = train_pat.iloc[batch_idx]
        batch_y = np.array(train_label)[batch_idx]
        fd = {x:batch_x, y:batch_y}
        
        sess.run(train, feed_dict=fd)
        if step % 500 == 0:
            cost, acc = sess.run([cross_entropy, accuracy], feed_dict=fd)
            
            print(cost, acc)
            
    predict = sess.run(tf.argmax(h, 1), feed_dict={x:batch_x})
    print("predict : ", predict)

    acc = sess.run(accuracy, feed_dict={x:test_pat, y:test_label})
    print("정확도 : ", acc)

1.1832767 0.28
0.606637 0.86
0.47534883 0.92
0.39659667 0.95
0.3539328 0.98
0.3464509 0.94
0.3285863 0.96
0.25925717 0.99
0.2585477 0.98
0.27061343 0.98
predict :  [2 0 1 0 0 2 1 0 2 1 1 0 2 1 0 0 0 2 0 2 1 0 0 1 1 2 0 1 2 2 1 0 2 0 1 0 2
 0 2 2 0 0 2 1 1 2 2 2 2 1 1 0 2 2 0 1 1 1 2 2 0 2 2 2 0 1 0 1 2 2 2 0 2 2
 2 2 2 0 2 2 2 0 0 0 2 0 2 2 2 0 0 0 1 2 2 2 0 2 0 0]
정확도 :  0.9788
